In [2]:
import pandas as pd
import numpy as np
import glob
from pandarallel import pandarallel
from bs4 import BeautifulSoup, SoupStrainer
import requests
import os
import random


In [21]:
os.makedirs("data/github_clean/linked_issues", exist_ok=True)

In [22]:
pd.set_option('display.max_columns', None)

In [23]:
def grabIssue(repo, issue_number):
    try:
        scrape_url = f"https://github.com/{repo}/issues/{issue_number}"
        product = SoupStrainer('a')
        sesh = requests.Session() 
        page = sesh.get(scrape_url)
        page_text = str(page.text)
        if "Please wait a few minutes before you try again" in page_text:
            print('pausing, rate limit hit')
            time.sleep(120)
            page = sesh.get(scrape_url)
            page_text = str(page.text)
            
        soup = BeautifulSoup(page.content,parse_only = product,features="html.parser")
        pr_links = soup.find_all("a", attrs={"data-hovercard-type":'pull_request'})
        pr_link = pr_links[0]['href']
        return pr_link
    except Exception as e:
        error = str(e)
        return error

In [13]:
files = glob.glob('data/inputs/linked_issues/*')

In [14]:
random.shuffle(files)

In [24]:
for file in files:
    check_issues = pd.read_csv(file, index_col = 0)
    fname = files[0].split("/")[-1]
    if fname in os.listdir('data/github_clean/linked_issues'):
        continue
    else:
        while check_issues.dropna().shape[0] != check_issues.shape[0]:
            na_df = check_issues[check_issues['linked_pr'].isna()]
            print(f"{na_df.shape[0]} remaining in {file}")
            # update file
            apply_inds = na_df.sample(min(10, na_df.shape[0])).index
            check_issues.loc[apply_inds, 'linked_pr'] = check_issues.loc[apply_inds].apply(
                lambda x: grabIssue(x['repo_name'], x['potential_issues']), axis = 1)
            ## update with new changes
            check_issues.to_csv(f'data/github_clean/linked_issues/{fname}')

356 remaining in data/inputs/linked_issues/linked_issue_855.csv
346 remaining in data/inputs/linked_issues/linked_issue_855.csv


KeyboardInterrupt: 

In [3]:
linked_issues = pd.concat([pd.read_csv(file, index_col = 0) for file in glob.glob('data/github_clean/linked_issues/*')])

In [4]:
linked_issues = linked_issues[linked_issues['linked_pr'] != 'list index out of range']

In [5]:
linked_issues['pr_number'] = linked_issues['linked_pr'].apply(lambda x: x.split("/")[-1])

In [6]:
linked_issues.reset_index(drop = True, inplace = True)

In [7]:
correct = linked_issues[linked_issues['pr_number'].apply(lambda x: '#' in x)].index
linked_issues.loc[correct, 'pr_number'] = linked_issues.loc[correct, 'pr_number'].apply(lambda x: x.split("#")[0])

In [8]:
linked_issues.to_csv('data/merged_data/linked_issues.csv')